https://www.yeastgenome.org/

https://www.yeastgenome.org/api/doc

https://www.cse.msu.edu/~cse835/Papers/Graph_connectivity_revised.pdf => connexite

Maslov et al., 2013, Segundo et al., 2011, Tomita and Kameda, 2007 => Ensemble stable


isthme, trouver les cycles et circonférence, sommet-connexite, centralité, nb chromatique, contraction plutôt que suppression, partition, dureté, force, stable, clique, noyau, dominants


In [40]:
import pandas as pd
import requests
#import networkx

In [41]:
def parcours_profondeur(graphe: dict, sommet: int) -> list:
    """
        Algorithme de parcours en profondeur d'un graphe.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            - sommet (int): correspond à la racine de l'aborescence de parcours en profondeur.
        
        Sorties:
            - pere (list): correspon à l'arborescence avec pere[x] est le père du sommet x dans le parcous en profondeur.
            
        Complexité:
            - O(n)
    """
    taille = max(graphe.keys())
    
    etat = [None for _ in range(taille)]
    pere = [None for _ in range(taille)]
    dernier_pred = [None for _ in range(taille)]
    
    nb_vu = 1
    
    pile = [sommet]
    
    while nb_vu <= taille:
        
        if nb_vu not in graphe.keys():
            
            nb_vu += 1
        
        elif etat[nb_vu - 1] is not None:
            
            nb_vu += 1
            
        elif pile == []:
            
            pile.append(nb_vu)
            
        while pile != []:
            
            suivant = pile.pop()
            
            if etat[suivant - 1] is None:
                
                etat[suivant - 1] = 0
                pere[suivant - 1] = dernier_pred[suivant - 1]
                
                for successeur in graphe[suivant]:
                    
                    pile.append(successeur)
                    dernier_pred[successeur - 1] = suivant
    return pere


def excentricite_sommet(graphe: dict, sommet: int) -> int:
    """
        Algorithme de calcul d'excentricité d'un sommet dans un graphe. Aussi la hauteur de l'aborescence d'un parcours
        en largeur du graphe à partir du sommet.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            - sommet (int): correspond à la racine de l'aborescence de parcours en profondeur.
        
        Sorties:
            - max(hauteur) (int): correspond à l'excentricité du sommet.
            
        Complexité:
            - O(n)
    """
    taille = max(graphe.keys())
    
    etat = [None for _ in range(taille)]
    hauteur = [0 for _ in range(taille)]
    
    file = [sommet]
    etat[sommet - 1] = 0
    
    nb_vu = 1
    
    while nb_vu <= taille:
        
        if nb_vu not in graphe.keys():
            
            nb_vu += 1
        
        elif etat[nb_vu - 1] is not None:
            
            nb_vu += 1
            
        elif file == []:
            
            file.append(nb_vu)
            etat[nb_vu - 1] = 0
            
        while file != []:
            
            suivant = file[0]
            file = file[1:] if len(file) > 1 else []
            
            for successeur in graphe[suivant]:
                
                if etat[successeur - 1] is None:
                    
                    file.append(successeur)
                    etat[successeur - 1] = 0
                    hauteur[successeur - 1] = hauteur[suivant - 1] + 1
                    
    return max(hauteur)


def composantes_connexes(graphe: dict) -> list:
    """
        Algorithme de calcul des composantes connexes d'un graphe par parcours en profondeur, à chaque fois qu'un sommet n'a
        pas était visité alors que la pile est vide, compose connexe + 1.
        L'algorithme renvoie de plus les composantes connexes sous forme de liste avec un dictionnaire pour représenter chaque
        composante connexe.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties: 
            - composante (list): Les composantes connexes du graphe.
            
        Complexité:
            - O(n).
    """
    taille = max(graphe.keys())
    
    etat = [None for _ in range(taille)]
    pere = [None for _ in range(taille)]
    dernier_pred = [None for _ in range(taille)]
    composante = []
    
    nb_vu = 1
    sous_graphe_connexe = {1: set()}
    
    pile = [1]
    
    while nb_vu <= taille:
        
        if nb_vu not in graphe.keys():
            
            nb_vu += 1
        
        elif etat[nb_vu - 1] is not None:
            
            nb_vu += 1
            
        elif pile == []:
            
            composante.append(sous_graphe_connexe)
            pile.append(nb_vu)
            sous_graphe_connexe = {nb_vu: set()}
            
        while pile != []:
            
            suivant = pile.pop()
            
            if etat[suivant - 1] is None:
                
                sous_graphe_connexe[suivant] = set()
                etat[suivant - 1] = 0
                pere[suivant - 1] = dernier_pred[suivant - 1]
                
                for successeur in graphe[suivant]:
                    
                    sous_graphe_connexe[successeur] = set() if etat[successeur - 1] is None else sous_graphe_connexe[successeur]
                    sous_graphe_connexe[suivant].add(successeur)
                    sous_graphe_connexe[successeur].add(suivant)
                    pile.append(successeur)
                    dernier_pred[successeur - 1] = suivant
                    
    composante.append(sous_graphe_connexe)
    
    return composante


def sommets_isoles(graphe: dict) -> set:
    """
        Algorithme qui renvoie les sommets isolées d'un graphe, ou les sommets de degré = 0.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            
        Sorties:
            - isoles (set): ensemble de sommets isolées i.e de degré nul.
            
        Complexité:
            - O(n)
    """
    isoles = set()
    
    for v1, voisins in graphe.items():
        
        if len(voisins) == 0:
            
            isoles = isoles.union({v1})
    
    return isoles


def cycle(graphe: dict) -> bool:
    """
        Algorithme qui réalise un parcours en profondeur, si rencontre deux fois un sommet alors il renvoie vrai 
        si le graphe possède un cycle, faux sinon.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - cyclique (bool): Vrai si le graphe possède un cycle, faux sinon.
            
        Complexité:
            - O(n)
    """
    cyclique = False
    
    taille = len(graphe.keys())
    
    etat = [None for _ in range(taille)]
    dernier_pred = [None for _ in range(taille)]
    
    nb_vu = 1
    
    pile = [list(graphe.keys())[0]]
    
    while nb_vu <= taille:
        
        if nb_vu not in graphe.keys():
            
            nb_vu += 1
        
        elif etat[nb_vu - 1] is not None:
            
            nb_vu += 1
            
        elif pile == []:
            
            pile.append(nb_vu)
            cc += 1
            
        while pile != []:
            
            suivant = pile.pop()
            
            if etat[suivant - 1] is None:
                
                etat[suivant - 1] = 0
                
                for successeur in graphe[suivant]:
                    
                    if (etat[successeur - 1] is not None) and (dernier_pred[suivant - 1] != successeur):
                        
                        cyclique = True
                        break
                        
                    pile.append(successeur)
                    dernier_pred[successeur - 1] = suivant
    
                if cyclique: break
            if cyclique: break
        if cyclique: break
    
    return cyclique


def points_articulations(graphe: dict) -> set:
    """
        Algorithme qui renvoie l'ensemble des points d'articulations d'un graphe, par un parcours en profondeur si la racine
        possède N fils avec N > 1 alors point d'articulation.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - points (set): L'ensemble des points d'articulations d'un graphe.
            
        Complexité:
            - O(n^2)
    """
    points = set()
    
    for s in graphe:
        points = points.union({s}) if parcours_profondeur(graphe, s).count(s) > 1 else points
    
    return points


articulation = lambda graphe, s: True if parcours_profondeur(graphe, s).count(s) > 1 else False


def supprimer_sommet(graphe: dict, sommet: int) -> dict:
    """
        Algorithme permettant la création d'un sous-graphe induit avec G' = (V', E') et V - {sommet}.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            - sommet (int): le sommet a supprimé.
        
        Sorties:
            - nouveau_graphe (dict): correspond au sous-graphe induit.
        
        Complexité:
            - O(|V|+|E|)
    """
    nouveau_graphe = graphe.copy()
    
    for s in nouveau_graphe.keys():
        nouveau_graphe[s] = nouveau_graphe[s].difference({sommet})
    del nouveau_graphe[sommet]
    
    return nouveau_graphe

def graphe_complementaire(graphe: dict) -> dict:
    """
        Algorithme qui permet de calculer le graphe complémentaire de G.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - complementaire (dict): graphe complémentaire du graphe d'entrée
        
        Complexité:
            - O(?)
    """
    complementaire = {sommet: set(graphe.keys()) for sommet in graphe.keys()}
    
    for s, voisins in graphe.items():
        
        complementaire[s] = complementaire[s].difference(graphe[s].union({s}))

    return complementaire

    
def centres(graphe: dict) -> tuple:
    """
        Algorithme calculant l'ensemble des centres d'un graphe par le calcul de l'excentricité minimale.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - (tuple): Renvoie (x, y) avec x l'ensemble des centres et y leurs excentricités.
            
        Complexité:
            - O(n^2)
    """
    excent = {sommet: 0 for sommet in graphe.keys()}
    
    for sommet in graphe:
        excent[sommet] = excentricite_sommet(graphe, sommet)
    
    return set(sommet for sommet in graphe if excent[sommet] == min(excent.values())), min(excent.values())


def diametre(graphe: dict) -> int:
    
    excent = {sommet: 0 for sommet in graphe.keys()}
    
    for sommet in graphe:
        excent[sommet] = excentricite_sommet(graphe, sommet)
    
    return set(sommet for sommet in graphe if excent[sommet] == max(excent.values())), max(excent.values())


densite = lambda graphe: sum(list(map(len, list(graphe.values()))))/(len(graphe.keys())*(len(graphe.keys()) - 1))


def sommets_couvrants(graphe: dict) -> set:
    
    """Chvatal algorithm à la place"""
    
    """
        Heuristique de calcul des sommets couvrants (vertex cover).
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - dom (set): l'ensemble (dont on espère minimum) des sommets couvrants l'ensemble des arêtes.
            
        Complexité:
            - O(n^2)
    """
    
    deg = {sommet: len(voisins) for sommet, voisins in graphe.items()}
    
    def procedure(graphe: dict, degres: dict, dom: set) -> set:
        
        if len(degres) == 0 or max(degres.values()) == 0:
            
            return dom
        
        nouveau = max(degres, key=lambda k: degres[k])
        
        dom = dom.union({nouveau})
        
        for sommet in degres:
            
            if nouveau in graphe[sommet]:
                
                degres[sommet] -= 1
                
        del degres[nouveau]
        
        return procedure(graphe, degres, dom)
    
    return procedure(graphe, deg, set())
    

def contraction(graphe: dict, s1: int, s2: int) -> dict:
    """
        Algorithme de contraction de deux sommets s1, s2 en un sommet appelé s1/s2.
        
        Variable:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
            - s1 (int): le sommet n°1 a fusionné.
            - s2 (int): le sommet n°2 a fusionné.
        
        Sorties:
            - nouveau_graphe (dict): le graphe contracté.
        
        Complexité:
            - O(?)
    """
    nouveau_graphe = graphe.copy()
    
    nouveau_graphe[f"{s1}/{s2}"] = nouveau_graphe[s1].union(graphe[s2])
    
    for sommet, voisins in graphe.items():
        if s1 in voisins or s2 in voisins:
            
            nouveau_graphe[sommet] = (nouveau_graphe[sommet].difference({s1, s2})).union({f"{s1}/{s2}"})
    
    del nouveau_graphe[s1]
    del nouveau_graphe[s2]
    
    return nouveau_graphe

def contraction_pondere(s1, s2, graphe):
    """
        Algorithme de contraction de deux sommets s1, s2 en un sommet appelé s1/s2 et pour toute arêtes incidentes à 
        s1 ou s2 on a w(s1/s2, x) = somme(w(s1, x), w(s2, x)).
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {(voisin1, poids1), (voisin2, poids2), ..., (voisinN, poidsN)}}.
            - s1 (int): le sommet n°1 a fusionné.
            - s2 (int): le sommet n°2 a fusionné.
        
        Sorties:
            - nouveau_graphe (dict): le graphe contracté.
        
        Complexité:
            - O(?)
    """
    sf = f"{s1}/{s2}"
    nouveau_graphe = graphe.copy()

    for sommet in graphe:
        nouveau_graphe[sommet] = {arete for arete in graphe[sommet] if arete[0] not in {s1, s2}}
    del nouveau_graphe[s1], nouveau_graphe[s2]

    for sommet in nouveau_graphe:
        nouveau_graphe[sommet] = nouveau_graphe[sommet].union({(sf, sum(list(map(lambda arete: arete[1], {art for art in graphe[sommet] if art[0] in {s1, s2}}))))})
        nouveau_graphe[sommet] = {(sommet, poids) for sommet, poids in nouveau_graphe[sommet] if poids > 0}
        
    nouveau_graphe[sf] = set()    
        
    for sommet in nouveau_graphe:
        nouveau_graphe[sf] = nouveau_graphe[sf].union({(sommet, arete[1]) for arete in nouveau_graphe[sommet] if arete[0] == sf}) 

    nouveau_graphe[sf] = nouveau_graphe[sf].union({(sf, sum(list(map(lambda arete: arete[1], {art for art in graphe[s1].union(graphe[s2]) if art[0] in {s1, s2}}))))})    
        
    return nouveau_graphe


def stoer_wagner(graphe: dict) -> int:
    """
        Algorithme de Stoer-Wagner afin de trouver la coupe minimale d'un graphe i.e le k pour k-arêtes-connexes.
        
        Variables:
            - graphe (dict): graphe sous la forme de dictionnaire {sommet: {voisin1, voisin2, ..., voisinN}}.
        
        Sorties:
            - (int): La coupe minimale du graphe.
        
        Complexité:
            - O(n^2)
    """
    nouveau_graphe = {sommet: {(voisin, 1) for voisin in voisins} for sommet, voisins in graphe.items()}
    
    coupe = set()
    
    while len(nouveau_graphe) > 2:
        
        graphe_poids = nouveau_graphe.copy()
        ensemble = list(sommet for sommet in nouveau_graphe if "/" in str(sommet))
        ensemble = 1 if ensemble == [] else ensemble[0]
        cpt = 0
        sommets = []
        
        while cpt < len(graphe_poids):
            
            s1, s2, w = None, None, -1
                
            v1 = ensemble
            v2, maximum_local = list(graphe_poids[ensemble])[0]

            for arete in graphe_poids[ensemble]:
                if arete[1] > maximum_local:
                    v2, maximum_local = arete

            if maximum_local > w:
                s1, s2, w = v1, v2, maximum_local
                    
            ensemble = f"{s1}/{s2}"
            sommets.append(s2)
            
            graphe_poids = contraction_pondere(s1, s2, graphe_poids)
            
            cpt += 1
            
        s1, s2 = sommets[-2:]
        
        s1 = str(s1) if "/" in str(s1) else int(s1)
        s2 = str(s2) if "/" in str(s2) else int(s2)
        
        nouveau_graphe = contraction_pondere(s1, s2, nouveau_graphe)
        coupe = coupe.union({w})
    
    coupe = coupe.union({max({poids for aretes in nouveau_graphe.values() for _, poids in aretes})})
    
    return min(coupe)
        

In [42]:
interactome = pd.read_csv("Saccharomyces cerevisiae/interactome.txt", sep="\t")
interactome

,P1,P2
0,142,850399
1,142,850500
2,142,851572
3,142,853494
4,142,855159
...,...,...
229691,9164967,856237
229692,9164967,856459
229693,9164977,850883
229694,9164990,851529


In [43]:
def TrouveCommunaute(communautes, i):
    ''' Renvoie le numero de la communaute du noeud i'''
    
    return communautes[i]

def QuiDansCommunaute(communautes, C):
    ''' Renvoie une liste des noeuds appartenant la communauté C'''

    commu = []
    taille = len(communautes)
    for n in range(taille):
        if communautes[n] == C:
            commu.append(n)
    
    return commu

def TrouveVoisins(dico, i):
    ''' Renvoie une liste des voisins du sommet i'''
    
    l_res = []
    v = dico[i]

    for j in v:
        l_res.append(j[0])

    return l_res


def AjoutDansCommunaute(communautes, i, C):
    ''' Ajoute i à la communauté C dans la liste communautés '''
    # communautes est une liste où chaque élément d'indice i est le numéro de la communauté de i

    communautes[i] = C
    return communautes

def renommeCommu(commu):
    set_commu = list(set(commu))

    li_correspondance = [i for i in range(len(set_commu))]

    taille_commu=len(commu)
    for i in range(taille_commu):
        taille_set_commu = len(set_commu)
        for j in range(taille_set_commu):
            if commu[i] == set_commu[j]:
                commu[i] = li_correspondance[j]

    return commu


def Calcul_m(dico):
    ''' Calcule la somme des poids du graphe '''
    
    somme_totale = 0
    somme_intermediaire = 0

    for sommet, voisins in dico.items():
        for v in voisins:
            if sommet != v[0]:                      # si ce n'est pas une boucle => on la comptera deux fois
                somme_intermediaire += v[1]
            else:                                   # si c'est une boucle
                somme_totale += v[1]

    somme_totale += somme_intermediaire//2

    return somme_totale  

def Calcul_ki(dico, i):
    ''' Calcule la somme des poids des arêtes du noeud i'''

    ki = 0

    for v in dico[i]:
        ki += v[1]

    return ki

def Calcul_S_tot(dico, noeuds_dans_commu, i):
    ''' Calcule la somme des poids des arêtes de la communauté testée '''
    # noeuds_dans_commu est une liste contenant les numéros des noeuds qui sont dans cette communauté
    # i est le noeud testé

    S_tot = 0

    for n in noeuds_dans_commu:
        if n != i:
            S_tot += Calcul_ki(dico, n)

    return S_tot

def Calcul_ki_in(dico, communautes, i, C):
    ''' Calcule la somme des poids des arêtes de i vers la communauté C'''

    ki_in = 0
    voisins = dico[i]

    for v in voisins:
        if communautes[v[0]] == C:
            ki_in += v[1]
    
    return ki_in


def Louvain_P1(dico):
    
    flag_modif = True
    m = Calcul_m(dico)

    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]
    historique_communautes = []
    historique_communautes.append(communautes.copy())

    while flag_modif == True:       # tant qu'on modifie encore les communautés
        flag_modif = False

        taille_commu = len(communautes)
        for noeud in range(taille_commu-1, -1, -1):       # pour chaque noeud
            print(noeud)
            liste_voisins = TrouveVoisins(dico, noeud)
            max = -1
            max_v = -1
            
            liste_commu_voisins = []
            nv_liste_voisins = []
            for vo in liste_voisins:            # pour chaque voisin
                if communautes[vo] not in liste_commu_voisins:      # si la communauté n'est pas déjà dedans, on la rajoute
                    nv_liste_voisins.append(vo)
                    liste_commu_voisins.append(communautes[vo])         # ajoute dans liste_commu_voisins le numéro de la communauté de ce voisin

            liste_voisins = nv_liste_voisins    # permet de diminuer le nombre de calculs à faire

            # Calcul de ki
            ki = Calcul_ki(dico, noeud)

            for v in liste_voisins:     # pour chaque voisin du noeud

                if v != noeud:
                    n_commu_v = QuiDansCommunaute(communautes, communautes[v])

                    # Calcul de ki_in
                    ki_in = Calcul_ki_in(dico, communautes, noeud, communautes[v])

                    # Calcul de S tot
                    S_tot = Calcul_S_tot(dico, n_commu_v, noeud)

                    # Calcul du gain
                    gain = (1/(2*m))*((2*ki_in) - (S_tot * ki)/m)

                    if gain > max:      # si c'est un meilleur gain
                        max = gain
                        max_v = v
            
            #print(noeud, max, max_v)
            if (communautes[noeud] != communautes[max_v]) and (max > 0):        # si on doit mettre le noeud dans une communauté différente
                AjoutDansCommunaute(communautes, noeud, communautes[max_v])     # on le met dans cette communauté en question
                flag_modif = True       # on a modifié

        if communautes not in historique_communautes:
            historique_communautes.append(communautes.copy())
        else : 
            break

    return renommeCommu(communautes)
                        
    
def Louvain_P2(dico_graphe, communautes):
    '''Fonction qui réalise la seconde partie de l'algorithme de Louvain'''

    nouveau_graphe = {}
    nb_commu = len(set(communautes))

    for elt in range(nb_commu):
        li_sommet = []    
        taille_commu = len(communautes)                      #récupère les sommets d'une communauté
        for sommet in range(taille_commu):
            if communautes[sommet] == elt :
                li_sommet.append(sommet)

        poids_sommet_commu = 0                  #variable qui va contenir le poids du sommet (somme des aretes au sein de la commu)
        dico_arete_autre_commu = {}             #dico qui va contenir le nb d'aretes d'une commu vers chaque autres commu qui va correspondre au poids de l'arete reliant l'autre commu
        
        for sommet_commu in li_sommet :
            for arete in dico_graphe[sommet_commu]: #parcours les sommets de la commu en cours de traitement
                if arete[0] in li_sommet :       #parcours les arete de chaque sommet de la commu
                    poids_sommet_commu += arete[1]  #si le sommet relié au sommet en cours de traitement est aussi dans la commu on ajoute le poids de l'arete dans notre variable
                else:
                    if communautes[arete[0]] not in dico_arete_autre_commu :    #si il n'est pas dans la commu
                        dico_arete_autre_commu[communautes[arete[0]]] = arete[1] #si c'est le premier/seul elt connecté à notre commu on ajoute dans le dico la commu associé à son poids
                    else :
                        dico_arete_autre_commu[communautes[arete[0]]] += arete[1] #sinon on additionne le poids de la nouvelle arete reliant la commu
        
        for commu, poids in dico_arete_autre_commu.items():     #partie permettant de creer le nouveau graphe
            if elt not in nouveau_graphe :                      #si on n'a pas encore traité la commu
                nouveau_graphe[elt]=[(elt, poids_sommet_commu)] #on ajoute l'arete qui va sur elle meme permettant d'indiquer le poids du sommet
                nouveau_graphe[elt].append((commu, poids))      #on ajoute ensuite les aretes vers l'autre commu en cours de traitement
            else : 
                nouveau_graphe[elt].append((commu, poids))      #on a déjà la commu dans le nouveau graphe, ça veut donc dire qu'on traite les commu suivante vers laquelle il est relié

    return nouveau_graphe, communautes


def Louvain_Final(dico, nom_fichier):
    ''' Appelle les 2 parties de l'algorithme de Louvain itérativement'''

    nouveau_graphe = dico
    historique_graphes = [nouveau_graphe]
    taille_graphe = len(dico)
    historique_commu = [[k for k in range(taille_graphe)]]
    c = 0
    while True:
        print('PASSE ', c)
        nvelles_communautes = Louvain_P1(nouveau_graphe)
        nouveau_graphe, new_commu = Louvain_P2(nouveau_graphe, nvelles_communautes)
        
        if len(nouveau_graphe) == len(historique_graphes[-1]):
            break

        historique_graphes.append(nouveau_graphe)
        historique_commu.append(new_commu)
        print('une passe a été faite')
        c += 1

    historique_commu.append(new_commu)
    with open(nom_fichier, 'w') as fic:
        fic.write(str(historique_graphes) + '\n')
        fic.write(str(historique_commu))

    return 'fin'

In [44]:
sommets = {g for g in set(interactome["P1"]).union(set(interactome["P2"]))}
g_to_i = {g: i for i, g in enumerate(sommets)}
i_to_g = {i: g for i, g in zip(g_to_i.values(), g_to_i.keys())}
G = {i: list() for i in i_to_g.keys()}
print(i_to_g[0], g_to_i[851968])

851968 0


In [45]:
with open("g_to_i.txt", "w") as filout:
    texte = ""
    for g, i in g_to_i.items():
        texte += f"{g} {i}\n"
    filout.write(texte)

In [46]:
for interaction in interactome.itertuples():
    if interaction.P1 != interaction.P2:
        G[g_to_i[interaction.P1]].append((g_to_i[interaction.P2], 1))
        G[g_to_i[interaction.P2]].append((g_to_i[interaction.P1], 1))
print(G[0])

[(5125, 1), (5346, 1), (5519, 1), (5809, 1), (5858, 1), (5955, 1), (6258, 1), (5129, 1), (5355, 1), (5648, 1), (5658, 1), (5737, 1), (6000, 1), (6197, 1), (11, 1), (56, 1), (347, 1), (416, 1), (692, 1), (710, 1), (757, 1), (772, 1), (783, 1), (833, 1), (862, 1), (900, 1), (956, 1), (1018, 1), (1105, 1), (1187, 1), (1235, 1), (1236, 1), (1253, 1), (1304, 1), (1572, 1), (1781, 1), (1884, 1), (2041, 1), (2605, 1), (2801, 1), (2847, 1), (2937, 1), (2994, 1), (3076, 1), (3174, 1), (3453, 1), (3551, 1), (3598, 1), (3880, 1), (4137, 1), (4145, 1), (4280, 1), (4371, 1), (7, 1), (114, 1), (153, 1), (745, 1), (1184, 1), (1602, 1), (2505, 1), (2940, 1), (3136, 1), (3382, 1), (3403, 1), (3518, 1), (3815, 1), (4299, 1), (4412, 1)]


In [47]:
print(G[1523])

[(4540, 1), (5085, 1), (5173, 1), (5316, 1), (5369, 1), (5544, 1), (5549, 1), (6036, 1), (6056, 1), (6104, 1), (6157, 1), (6257, 1), (6355, 1), (6503, 1), (146, 1), (368, 1), (526, 1), (618, 1), (711, 1), (722, 1), (732, 1), (929, 1), (957, 1), (1044, 1), (1361, 1), (1153, 1), (3998, 1), (4258, 1), (4824, 1), (4832, 1), (1643, 1), (1686, 1), (1758, 1), (1776, 1), (2398, 1), (2517, 1), (2546, 1), (2628, 1), (2651, 1), (2751, 1), (2806, 1), (2909, 1), (2953, 1), (2974, 1), (3060, 1), (3089, 1), (3131, 1), (3174, 1), (3247, 1), (3371, 1), (3422, 1), (3424, 1), (3469, 1), (3521, 1), (3634, 1), (3712, 1), (3858, 1), (3936, 1), (4023, 1), (4037, 1), (4106, 1), (4142, 1), (4164, 1), (4168, 1), (4306, 1), (4431, 1)]


In [48]:
#print(Louvain_Final(G, 'Result_AlgoDecroissant.txt'))

In [49]:
# Le graphe test contient 21 sommets et 34 arêtes

dico_test = {0:[(1,1)],
            1:[(0,1), (2,1)],
            2:[(1,1), (3,1), (4,1)],
            3:[(2,1), (4,1), (8,1)],
            4:[(2,1), (3,1), (5,1), (7,1)],
            5:[(4,1), (6,1), (17,1)],
            6:[(5,1), (7,1), (19,1)],
            7:[(4,1), (6,1), (9,1)],
            8:[(3,1)],
            9:[(7,1), (10,1), (11,1)],
            10:[(9,1), (11,1)],
            11:[(9,1), (10,1), (12,1)],
            12:[(11,1), (14,1), (17,1), (19,1)],
            13:[(14,1), (15,1), (16,1), (19,1)],
            14:[(12,1), (13,1), (15,1)],
            15:[(13,1), (14,1), (16,1), (17,1)],
            16:[(13,1), (15,1), (17,1), (18,1), (20,1)],
            17:[(5,1), (12,1), (15,1), (16,1), (18,1)],
            18:[(16,1), (17,1), (19,1), (20,1)],
            19:[(6,1), (12,1), (13,1), (18,1), (20,1)],
            20:[(16,1), (18,1), (19,1)]
            }

In [50]:
dico_SV = {0:[(2,1), (3,1), (4,1), (5,1)],
            1:[(2,1), (4,1), (7,1)],
            2:[(0,1), (1,1), (4,1), (5,1), (6,1),],
            3:[(0,1), (7,1)],
            4:[(0,1), (1,1), (2,1), (10,1)],
            5:[(0,1), (2,1), (7,1), (11,1)],
            6:[(2,1), (7,1), (11,1)],
            7:[(1,1), (3,1), (5,1), (6,1)],
            8:[(9,1), (10,1), (11,1), (14,1), (15,1)],
            9:[(8,1), (12,1), (14,1)],
            10:[(4,1), (8,1), (11,1), (12,1), (13,1), (14,1)],
            11:[(5,1), (6,1), (8,1), (10,1), (13,1)],
            12:[(9,1), (10,1)],
            13:[(10,1), (11,1)],
            14:[(8,1), (9,1), (10,1)],
            15:[(8,1)]}

In [51]:
print(Louvain_Final(dico_SV, 'testAlgo3.txt'))

PASSE  0
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
une passe a été faite
PASSE  1
3
2
1
0
3
2
1
0
une passe a été faite
PASSE  2
1
0
fin


In [52]:
print(Louvain_Final(dico_test, 'testAlgo2.txt'))

PASSE  0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
une passe a été faite
PASSE  1
4
3
2
1
0
4
3
2
1
0
4
3
2
1
0
4
3
2
1
0
une passe a été faite
PASSE  2
1
0
fin


In [53]:
graphe30 ={
    0: [(1, 1), (2, 1), (3, 1), (4, 1), (5, 1)],
    1: [(0, 1), (2, 1), (5, 1), (6, 1)],
    2: [(0, 1), (1, 1), (3, 1), (6, 1), (7, 1)],
    3: [(0, 1), (2, 1), (4, 1), (7, 1)],
    4: [(0, 1), (3, 1), (5, 1), (8, 1)],
    5: [(0, 1), (1, 1), (4, 1), (6, 1), (8, 1), (9, 1)],
    6: [(1, 1), (2, 1), (5, 1), (7, 1), (9, 1), (10, 1)],
    7: [(2, 1), (3, 1), (6, 1), (10, 1)],
    8: [(4, 1), (5, 1), (9, 1), (11, 1)],
    9: [(5, 1), (6, 1), (8, 1), (10, 1), (11, 1), (12, 1)],
    10: [(6, 1), (7, 1), (9, 1), (12, 1)],
    11: [(8, 1), (9, 1), (12, 1), (13, 1)],
    12: [(9, 1), (10, 1), (11, 1), (13, 1)],
    13: [(11, 1), (12, 1), (14, 1)],
    14: [(13, 1), (15, 1), (16, 1)],
    15: [(14, 1), (16, 1), (17, 1)],
    16: [(14, 1), (15, 1), (17, 1)],
    17: [(15, 1), (16, 1), (18, 1)],
    18: [(17, 1), (19, 1), (20, 1)],
    19: [(18, 1), (20, 1), (21, 1)],
    20: [(18, 1), (19, 1), (22, 1)],
    21: [(19, 1), (22, 1), (23, 1)],
    22: [(20, 1), (21, 1), (24, 1)],
    23: [(21, 1), (24, 1), (25, 1)],
    24: [(22, 1), (23, 1), (26, 1)],
    25: [(23, 1), (26, 1), (27, 1)],
    26: [(24, 1), (25, 1), (28, 1)],
    27: [(25, 1), (28, 1), (29, 1)],
    28: [(26, 1), (27, 1)],
    29: [(27, 1)]
}

In [54]:
print(Louvain_Final(graphe30, 'testGraphe30.txt'))

PASSE  0
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
29
28
27
26
25
24
23
22
21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
une passe a été faite
PASSE  1
8
7
6
5
4
3
2
1
0
8
7
6
5
4
3
2
1
0
8
7
6
5
4
3
2
1
0
8
7
6
5
4
3
2
1
0
8
7
6
5
4
3
2
1
0
une passe a été faite
PASSE  2
4
3
2
1
0
4
3
2
1
0
4
3
2
1
0
une passe a été faite
PASSE  3
3
2
1
0
3
2
1
0
une passe a été faite
PASSE  4
2
1
0
fin


In [55]:
print(Louvain_Final(dico_SV, 'testGrapheSV.txt'))

PASSE  0
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0
une passe a été faite
PASSE  1
3
2
1
0
3
2
1
0
une passe a été faite
PASSE  2
1
0
fin


In [85]:
def Louvain_P1(dico:dict)->list:
    '''Fonction qui réalise la première partie de l'algorithme de Louvain
    
        Variables : 
            - dico : graphe sous la forme d'un dictionnaire {sommet1 : [(voisin1, poids), (voisin2, poids) ..., (voisinN, poids)], sommet2 : [...], sommetN : [...]}
        
        Sortie : 
            - communautes : correspond à la liste des communautés attribuées aux sommets du graphes 
                            -> chaque indice du graphe correspond au numéro du sommet et la valeur à cet indice est sa communauté
    '''
    
    flag_modif = True
    m = Calcul_m(dico)      # Calul du m utilisé dans la formule du gain de modularité 

    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]
    historique_communautes = []
    historique_communautes.append(communautes.copy())

    while flag_modif == True:       # tant qu'on modifie encore les communautés et que la liste des communautés est différente des précédentes
        flag_modif = False

        taille_commu = len(communautes)
        for noeud in range(taille_commu):       # Parcours chaque noeud du graphe
            liste_voisins = TrouveVoisins(dico, noeud)  # Récupère les voisins du noeud 
            
            max = -1        # Sommet pour lequel on maximise le gain
            max_v = -1      # Valeur de la modularité max obtenue
            
            liste_commu_voisins = []    # Va contenir les numéros des communautés différentes des voisins du noeud
            nv_liste_voisins = []       # Creation d'une nouvelle liste des voisins pour avoir seulement 1 voisin de chaque communauté voisine

            for vo in liste_voisins:    # pour chaque voisin
                if communautes[vo] not in liste_commu_voisins:
                    nv_liste_voisins.append(vo)                         # Si la communauté n'est pas dans la liste on rajoute le sommet
                    liste_commu_voisins.append(communautes[vo])         # Ajoute le numéro de la communauté de ce voisin
            
            # Nouvelle liste des voisins pour lesquels on va effectuer les calculs de gain de modularité afin de diminuer le temps de calcul
            liste_voisins = nv_liste_voisins    

            ki = Calcul_ki(dico, noeud)    # Calcul de ki

            for v in liste_voisins:     # Pour chaque voisin du noeud de la nouvelle liste de voisins

                if v != noeud:
                    n_commu_v = QuiDansCommunaute(communautes, communautes[v])   # Récupère la liste des noeuds appartenant à la communauté du voisin

                    # Calcul de ki_in
                    ki_in = Calcul_ki_in(dico, communautes, noeud, communautes[v])

                    # Calcul de S tot
                    S_tot = Calcul_S_tot(dico, n_commu_v, noeud)

                    # Calcul du gain
                    gain = (1/(2*m))*((2*ki_in) - (S_tot * ki)/m)

                    if gain > max:      # Si c'est un meilleur gain
                        max = gain
                        max_v = v
                        
            if (communautes[noeud] != communautes[max_v]) and (max > 0):        # Conditions pour changer le noeud de communauté
                AjoutDansCommunaute(communautes, noeud, communautes[max_v])
                flag_modif = True       # Modifaction de la variable pour dire qu'il y a eu modification 

        # Vérifie que la liste de communautés n'a pas déjà été établis précdemment et qu'on ne tourne pas en boucle
        if communautes not in historique_communautes:
            historique_communautes.append(communautes.copy())
        else : 
            break

    return renommeCommu(communautes)
                        
    
def Louvain_P2(dico_graphe : dict, communautes : list):
    '''Fonction qui réalise la seconde partie de l'algorithme de Louvain

        Variables : 
            - dico_graphe : graphe sous la forme d'un dictionnaire {sommet1 : [(voisin1, poids), (voisin2, poids) ..., (voisinN, poids)], sommet2 : [...], sommetN : [...]}
            - communautes : correspond à la liste des communautés attribuées aux sommets du graphes

        Sortie : 
            - nouveau_graphe : graphe sous forme d'un dictionnaire correspondant à la réduction du graphe de départ en fonction des communautés données
    '''

    nouveau_graphe = {} 
    nb_commu = len(set(communautes))

    for elt in range(nb_commu):     # Pour chaque communauté on récupère les sommets qui sont dans cette communauté
        li_sommet = []
        taille_commu = len(communautes)
        for sommet in range(taille_commu):
            if communautes[sommet] == elt :
                li_sommet.append(sommet)

        poids_sommet_commu = 0          # Va contenir le poids de l'arete boucle du sommet (somme des aretes au sein de la commu)
        dico_arete_autre_commu = {}     # Va contenir la somme des poids des arêtes allant dans chaque communauté différente de celle en cours de traitement
        
        for sommet_commu in li_sommet :                 # Pour chaque sommet de la communauté
            for arete in dico_graphe[sommet_commu]:     # Pour toutes les aretes du sommet
                if arete[0] in li_sommet :              # Si le voisin est dans la même communauté
                    poids_sommet_commu += arete[1]      # Ajoute le poids de l'arete
                
                else: # Sinon on va sommer le poids des aretes allant vers les autres communautés
                    if communautes[arete[0]] not in dico_arete_autre_commu :
                        dico_arete_autre_commu[communautes[arete[0]]] = arete[1]
                    else :
                        dico_arete_autre_commu[communautes[arete[0]]] += arete[1] 
        
        # Creation du nouveau graphe dans lequel on va ajouter le sommet correspondant à la communauté 'elt' et les arêtes vers les autres communautés
        for commu, poids in dico_arete_autre_commu.items():
            if elt not in nouveau_graphe :
                nouveau_graphe[elt]=[(elt, poids_sommet_commu)] # Ajout du sommet correspond à la communauté elt et de son arete boucle
                nouveau_graphe[elt].append((commu, poids))
            else : 
                nouveau_graphe[elt].append((commu, poids))

    return nouveau_graphe


def Louvain_Final1(dico, nom_fichier):
    ''' Appelle les 2 parties de l'algorithme de Louvain itérativement
    
        Variables : 
                - dico_graphe : graphe sous la forme d'un dictionnaire {sommet1 : [(voisin1, poids), (voisin2, poids) ..., (voisinN, poids)], sommet2 : [...], sommetN : [...]}
                - communautes : correspond à la liste des communautés attribuées aux sommets du graphes

            Sortie : 
                - nouveau_graphe : graphe sous forme d'un dictionnaire correspondant à la réduction du graphe de départ en fonction des communautés données
                - communautes : 
    '''

    nouveau_graphe = dico
    historique_graphes = [nouveau_graphe]
    taille_graphe = len(dico)
    historique_commu = [[k for k in range(taille_graphe)]]
    c = 0
    while True:
        print('PASSE ', c)
        nvelles_communautes = Louvain_P1(nouveau_graphe)
        nouveau_graphe = Louvain_P2(nouveau_graphe, nvelles_communautes)
        
        if len(nouveau_graphe) == len(historique_graphes[-1]):
            break

        historique_graphes.append(nouveau_graphe)
        historique_commu.append(nvelles_communautes)
        print('une passe a été faite')
        c += 1

    historique_commu.append(nvelles_communautes)
    with open(nom_fichier, 'w') as fic:
        fic.write(str(historique_graphes) + '\n')
        fic.write(str(historique_commu))

    return 'fin'

In [87]:
def Louvain_P1(dico:dict)->list:
    '''Fonction qui réalise la première partie de l'algorithme de Louvain
    
        Variables : 
            - dico : graphe sous la forme d'un dictionnaire {sommet1 : [(voisin1, poids), (voisin2, poids) ..., (voisinN, poids)], sommet2 : [...], sommetN : [...]}
        
        Sortie : 
            - communautes : correspond à la liste des communautés attribuées aux sommets du graphes 
                            -> chaque indice du graphe correspond au numéro du sommet et la valeur à cet indice est sa communauté
    '''
    
    flag_modif = True
    m = Calcul_m(dico)      # Calul du m utilisé dans la formule du gain de modularité 

    # Initialisation de la liste communautes : chaque noeud est dans une communauté différente 
    communautes = [k for k in range(len(dico))]
    historique_communautes = []
    historique_communautes.append(communautes.copy())

    while flag_modif == True:       # tant qu'on modifie encore les communautés et que la liste des communautés est différente des précédentes
        flag_modif = False

        taille_commu = len(communautes)
        for noeud in range(taille_commu):       # Parcours chaque noeud du graphe
            liste_voisins = TrouveVoisins(dico, noeud)  # Récupère les voisins du noeud 
            
            max = -1        # Sommet pour lequel on maximise le gain
            max_v = -1      # Valeur de la modularité max obtenue
            
            liste_commu_voisins = []    # Va contenir les numéros des communautés différentes des voisins du noeud
            nv_liste_voisins = []       # Creation d'une nouvelle liste des voisins pour avoir seulement 1 voisin de chaque communauté voisine

            for vo in liste_voisins:    # pour chaque voisin
                if communautes[vo] not in liste_commu_voisins:
                    nv_liste_voisins.append(vo)                         # Si la communauté n'est pas dans la liste on rajoute le sommet
                    liste_commu_voisins.append(communautes[vo])         # Ajoute le numéro de la communauté de ce voisin
            
            # Nouvelle liste des voisins pour lesquels on va effectuer les calculs de gain de modularité afin de diminuer le temps de calcul
            liste_voisins = nv_liste_voisins    

            ki = Calcul_ki(dico, noeud)    # Calcul de ki

            for v in liste_voisins:     # Pour chaque voisin du noeud de la nouvelle liste de voisins

                if v != noeud:
                    n_commu_v = QuiDansCommunaute(communautes, communautes[v])   # Récupère la liste des noeuds appartenant à la communauté du voisin

                    # Calcul de ki_in
                    ki_in = Calcul_ki_in(dico, communautes, noeud, communautes[v])

                    # Calcul de S tot
                    S_tot = Calcul_S_tot(dico, n_commu_v, noeud)

                    # Calcul du gain
                    gain = (1/(2*m))*((2*ki_in) - (S_tot * ki)/m)

                    if gain > max:      # Si c'est un meilleur gain
                        max = gain
                        max_v = v
                        
            if (communautes[noeud] != communautes[max_v]) and (max > 0):        # Conditions pour changer le noeud de communauté
                AjoutDansCommunaute(communautes, noeud, communautes[max_v])
                flag_modif = True       # Modifaction de la variable pour dire qu'il y a eu modification 

        # Vérifie que la liste de communautés n'a pas déjà été établis précdemment et qu'on ne tourne pas en boucle
        if communautes not in historique_communautes:
            historique_communautes.append(communautes.copy())
        else : 
            break

    return renommeCommu(communautes)
                        
    
def Louvain_P2(dico_graphe : dict, communautes : list)->dict:
    '''Fonction qui réalise la seconde partie de l'algorithme de Louvain

        Variables : 
            - dico_graphe : graphe sous la forme d'un dictionnaire {sommet1 : [(voisin1, poids), (voisin2, poids) ..., (voisinN, poids)], sommet2 : [...], sommetN : [...]}
            - communautes : correspond à la liste des communautés attribuées aux sommets du graphes

        Sortie : 
            - nouveau_graphe : graphe sous forme d'un dictionnaire correspondant à la réduction du graphe de départ en fonction des communautés données
    '''

    nouveau_graphe = {} 
    nb_commu = len(set(communautes))

    for elt in range(nb_commu):     # Pour chaque communauté on récupère les sommets qui sont dans cette communauté
        li_sommet = []
        taille_commu = len(communautes)
        for sommet in range(taille_commu):
            if communautes[sommet] == elt :
                li_sommet.append(sommet)

        poids_sommet_commu = 0          # Va contenir le poids de l'arete boucle du sommet (somme des aretes au sein de la commu)
        dico_arete_autre_commu = {}     # Va contenir la somme des poids des arêtes allant dans chaque communauté différente de celle en cours de traitement
        
        for sommet_commu in li_sommet :                 # Pour chaque sommet de la communauté
            for arete in dico_graphe[sommet_commu]:     # Pour toutes les aretes du sommet
                if arete[0] in li_sommet :              # Si le voisin est dans la même communauté
                    poids_sommet_commu += arete[1]      # Ajoute le poids de l'arete
                
                else: # Sinon on va sommer le poids des aretes allant vers les autres communautés
                    if communautes[arete[0]] not in dico_arete_autre_commu :
                        dico_arete_autre_commu[communautes[arete[0]]] = arete[1]
                    else :
                        dico_arete_autre_commu[communautes[arete[0]]] += arete[1] 
        
        # Creation du nouveau graphe dans lequel on va ajouter le sommet correspondant à la communauté 'elt' et les arêtes vers les autres communautés
        for commu, poids in dico_arete_autre_commu.items():
            if elt not in nouveau_graphe :
                nouveau_graphe[elt]=[(elt, poids_sommet_commu)] # Ajout du sommet correspond à la communauté elt et de son arete boucle
                nouveau_graphe[elt].append((commu, poids))
            else : 
                nouveau_graphe[elt].append((commu, poids))

    return nouveau_graphe


def Louvain_Final(dico, nom_fichier):
    ''' Appelle les 2 parties de l'algorithme de Louvain itérativement
    
        Variables : 
            - dico : graphe sous la forme d'un dictionnaire {sommet1 : [(voisin1, poids), (voisin2, poids) ..., (voisinN, poids)], sommet2 : [...], sommetN : [...]}
            - nom_fichier : correspond au nom du fichier de sortie

        Sortie : 
            - Fichier txt dans lequel sera écrit l'historique des graphes et des communautés après les différentes passes de l'algorithme
    '''

    nouveau_graphe = dico
    historique_graphes = [nouveau_graphe]
    taille_graphe = len(dico)
    historique_commu = [[k for k in range(taille_graphe)]]

    while True:
        nvelles_communautes = Louvain_P1(nouveau_graphe)
        nouveau_graphe = Louvain_P2(nouveau_graphe, nvelles_communautes)
        
        # Condition d'arrêt (taille du nouveau graphe doit être différente du précédent)
        if len(nouveau_graphe) == len(historique_graphes[-1]):
            break

        historique_graphes.append(nouveau_graphe)
        historique_commu.append(nvelles_communautes)

    # Ecriture des résultats dans le fichier de sortie
    historique_commu.append(nvelles_communautes)
    with open(nom_fichier, 'w') as fic:
        fic.write(str(historique_graphes) + '\n')
        fic.write(str(historique_commu))

    return "Fin de l'exécution de l'algorithme"

In [88]:
print(Louvain_Final(dico_SV, 'testGrapheSVAvecModif.txt'))

Fin de l'exécution de l'algorithme
